## Obtaining the predictors
In this case we will bring all the variables to start working with everything.
### The fast forward way!
## Does not includes yet the population. Worth trying in the future. Just include WorldPop in the list of Models.


In [1]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph,Kingdom,Occurrence
from drivers.graph_models import Cell,Mex4km, countObjectsOf
from drivers.graph_models import pickNode
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import numpy as np

## Use the ggplot style
plt.style.use('ggplot')

In [2]:
from raster_api.tools import RasterData
from raster_api.models import raster_models_dic as models
from sketches.models import Country
from mesh.models import MexMesh

Mexico = Country.objects.filter(name__contains="exico").get()
mexican_cells = MexMesh.objects.filter(cell__intersects=Mexico.geom)

### Obtaining everything

In [3]:
datadict = { key : RasterData(models[key],border=Mexico.geom) for key,value in models.iteritems()}
## Without resamling (whole data)
pixel_size = 0.25


In [4]:
datadict

{'DistanceToRoadMex': <raster_api.tools.RasterData at 0x7f87c38d4c50>,
 'Elevation': <raster_api.tools.RasterData at 0x7f87c38d4690>,
 'MaxTemperature': <raster_api.tools.RasterData at 0x7f87c38d4a90>,
 'MeanTemperature': <raster_api.tools.RasterData at 0x7f87c38d47d0>,
 'MinTemperature': <raster_api.tools.RasterData at 0x7f87c38d48d0>,
 'Precipitation': <raster_api.tools.RasterData at 0x7f87c38d4650>,
 'SolarRadiation': <raster_api.tools.RasterData at 0x7f88081f33d0>,
 'Vapor': <raster_api.tools.RasterData at 0x7f87c3c5dd10>,
 'WindSpeed': <raster_api.tools.RasterData at 0x7f8808190190>,
 'WorldPopLatam2010': <raster_api.tools.RasterData at 0x7f88081901d0>}

In [6]:
%time datacube_field = map(lambda raster : raster.rescale(pixel_size,algorithm='Cubic'),datadict.itervalues())
datacube = datacube_field

CPU times: user 412 ms, sys: 12 ms, total: 424 ms
Wall time: 4min 46s


In [7]:
## Agregate by month
# Take mean of everything
cubes = map(lambda (k,v): np.mean(v.toNumpyArray(),axis=0), datadict.iteritems())

In [8]:
extractCoords = lambda (k,v) : v.getCoordinates() 
coords = extractCoords(datadict.iteritems().next())

In [9]:
## Oke I need a way to extract the dataframe, maybe aggregate it by mean 
dataframe_cube = map(lambda cube: pd.DataFrame(cube.flatten()),cubes)

In [10]:
datacube = pd.concat(dataframe_cube,axis=1)

In [11]:
datacube = pd.concat([datacube,coords],axis=1)

In [12]:
datacube.columns = datadict.keys() + list(coords.columns)

## Let´s save it for the posterity

In [13]:
datacube.to_csv('/outputs/sample_predictors_complete.csv')

In [14]:
datacube[:10]

,MaxTemperature,SolarRadiation,Elevation,MinTemperature,WorldPopLatam2010,WindSpeed,DistanceToRoadMex,MeanTemperature,Vapor,Precipitation,Longitude,Latitude
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-118.408333,14.475
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-118.158333,14.475
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-117.908333,14.475
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-117.658333,14.475
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-117.408333,14.475
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-117.158333,14.475
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.908333,14.475
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.658333,14.475
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.408333,14.475
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.158333,14.475
